In [40]:
!pip install pandas

In [41]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('AB_NYC_2019.csv')

# Display the first few rows
print(df.head())


     id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2    Elisabeth           Manhattan        Harlem  40.80902  -73.94190   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room    149               1                  9  20

In [42]:
# Get information about the DataFrame
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [43]:
# Check for missing values
print(df.isnull().sum())

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [44]:
# Fill missing values with 'unknown'
df['name'].fillna('unknown', inplace=True)
df['host_name'].fillna('unknown', inplace=True)
df['last_review'].fillna('01-01-2001', inplace=True)

In [45]:
# Calculate median of reviews_per_month column
median_reviews_per_month = df['reviews_per_month'].median()

# Fill missing values with median
df['reviews_per_month'].fillna(median_reviews_per_month, inplace=True)

In [46]:
# Check again for missing values in specific columns
print(df[['name', 'host_name', 'last_review', 'reviews_per_month']].isnull().sum())


name                 0
host_name            0
last_review          0
reviews_per_month    0
dtype: int64


In [47]:
# Remove duplicate records
df.drop_duplicates(inplace=True)

# Verify the number of duplicates removed
print("Number of duplicate records removed:", len(df) - len(df.drop_duplicates()))

Number of duplicate records removed: 0


In [48]:
# Standardize categorical data
df['neighbourhood_group'] = df['neighbourhood_group'].str.lower()
df['neighbourhood'] = df['neighbourhood'].str.strip()
df['neighbourhood']=df['neighbourhood'].str.lower()
df['room_type'] = df['room_type'].str.lower()

In [49]:
from sklearn.preprocessing import StandardScaler

In [50]:
scaler = StandardScaler()
numerical_columns = ['latitude', 'longitude', 'price', 'minimum_nights',
                     'number_of_reviews', 'calculated_host_listings_count',
                     'availability_365', 'reviews_per_month']  # Including reviews_per_month
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [51]:
# Handle date parsing with potential format variations
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce', infer_datetime_format=True)

# Check for rows where parsing failed (display them or handle as needed)
print("Rows with failed date parsing:")
print(df[df['last_review'].isnull()]['last_review'])

# Optionally, fill missing dates with a placeholder like 'unknown' if needed
df['last_review'].fillna('01-01-2001', inplace=True)

Rows with failed date parsing:
2       NaT
19      NaT
26      NaT
36      NaT
38      NaT
         ..
48890   NaT
48891   NaT
48892   NaT
48893   NaT
48894   NaT
Name: last_review, Length: 10052, dtype: datetime64[ns]


<ipython-input-51-e74c7bfebf72>:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce', infer_datetime_format=True)


In [52]:
# Function to detect and optionally remove outliers using IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
    # Optionally, you can remove outliers by filtering
    df_cleaned = df[~outliers]
    return df_cleaned

# Specify numerical columns for outlier detection
numerical_columns = ['price', 'minimum_nights', 'number_of_reviews',
                     'reviews_per_month', 'calculated_host_listings_count',
                     'availability_365']

# Apply outlier detection to each numerical column
for col in numerical_columns:
    df = remove_outliers(df, col)


In [53]:
# Display summary after outlier removal
print("Summary after outlier removal:")
print(df.describe())

Summary after outlier removal:
                 id       host_id      latitude     longitude         price  \
count  2.498200e+04  2.498200e+04  24982.000000  24982.000000  24982.000000   
mean   1.830857e+07  5.438802e+07     -0.002165     -0.046548     -0.134885   
min    5.022000e+03  2.571000e+03     -4.038526     -6.331760     -0.635934   
25%    9.535400e+06  7.161980e+06     -0.701572     -0.644125     -0.344452   
50%    1.868645e+07  2.479845e+07     -0.145497     -0.076921     -0.219531   
75%    2.680934e+07  7.240690e+07      0.634677      0.281806      0.013655   
max    3.648724e+07  2.743213e+08      3.376357      5.181954      0.750689   
std    1.023447e+07  6.786412e+07      0.989258      0.872504      0.275765   

       minimum_nights  number_of_reviews                    last_review  \
count    24982.000000       24982.000000                          24982   
mean        -0.199041          -0.318785  2014-06-15 03:56:26.726443008   
min         -0.293996          -

In [54]:
# Function to handle categorical outliers by frequency thresholding
def handle_categorical_outliers(df, column, threshold=0.01):
    # Calculate frequency of each category
    category_counts = df[column].value_counts(normalize=True)
    # Identify categories below the threshold
    infrequent_categories = category_counts[category_counts < threshold].index
    # Replace infrequent categories with 'Other'
    df[column] = df[column].apply(lambda x: 'Other' if x in infrequent_categories else x)
    return df

# Specify categorical columns for handling outliers
categorical_columns = ['neighbourhood_group', 'neighbourhood', 'room_type']

# Apply outlier handling to each categorical column
for col in categorical_columns:
    df = handle_categorical_outliers(df, col)

In [55]:
# Display summary after handling categorical outliers
print("Summary after handling categorical outliers:")
print(df[categorical_columns].value_counts(normalize=True))

Summary after handling categorical outliers:
neighbourhood_group  neighbourhood              room_type      
manhattan            Other                      entire home/apt    0.055920
brooklyn             williamsburg               private room       0.054039
                     Other                      entire home/apt    0.053158
                     williamsburg               entire home/apt    0.045633
                     bedford-stuyvesant         private room       0.044072
                                                                     ...   
                     prospect-lefferts gardens  shared room        0.000120
                     fort greene                shared room        0.000080
manhattan            west village               shared room        0.000040
brooklyn             park slope                 shared room        0.000040
Other                Other                      shared room        0.000040
Name: proportion, Length: 84, dtype: float64


In [56]:
# Save cleaned DataFrame to a new CSV file (optional)
df.to_csv('cleaned_data.csv', index=False)